In [12]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import time
from datetime import datetime
from citipy import citipy
from config import weather_api_key

In [13]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
coordinates = list(lat_lngs)

In [14]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

636

In [15]:
geocode_base_url = 'https://api.openweathermap.org/data/2.5/weather?units=Imperial&appid=' + weather_api_key 

In [16]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = geocode_base_url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        # city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

    # If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | mongui
Processing Record 2 of Set 1 | vardo
Processing Record 3 of Set 1 | mar del plata
Processing Record 4 of Set 1 | talara
Processing Record 5 of Set 1 | taunggyi
Processing Record 6 of Set 1 | cape town
Processing Record 7 of Set 1 | buzmeyin
Processing Record 8 of Set 1 | oussouye
Processing Record 9 of Set 1 | albany
Processing Record 10 of Set 1 | castro
Processing Record 11 of Set 1 | nanortalik
Processing Record 12 of Set 1 | prince rupert
Processing Record 13 of Set 1 | solovetskiy
City not found. Skipping...
Processing Record 14 of Set 1 | thompson
Processing Record 15 of Set 1 | barrow
Processing Record 16 of Set 1 | daytona beach
Processing Record 17 of Set 1 | longyearbyen
Processing Record 18 of Set 1 | rabo de peixe
Processing Record 19 of Set 1 | gizo
Processing Record 20 of Set 1 | butaritari
Processing Record 21 of Set 1 | erzin
Processing Record 22 of Set 1 | tuktoyaktuk
Proc

Processing Record 35 of Set 4 | beringovskiy
Processing Record 36 of Set 4 | sakakah
Processing Record 37 of Set 4 | turinsk
Processing Record 38 of Set 4 | amapa
Processing Record 39 of Set 4 | te anau
Processing Record 40 of Set 4 | svetlogorsk
Processing Record 41 of Set 4 | jacareacanga
Processing Record 42 of Set 4 | alyangula
Processing Record 43 of Set 4 | nago
Processing Record 44 of Set 4 | rocha
Processing Record 45 of Set 4 | donskoye
Processing Record 46 of Set 4 | ostrovnoy
Processing Record 47 of Set 4 | saint-pierre
Processing Record 48 of Set 4 | sinnamary
Processing Record 49 of Set 4 | zima
Processing Record 50 of Set 4 | havelock
Processing Record 1 of Set 5 | tautira
Processing Record 2 of Set 5 | hasaki
Processing Record 3 of Set 5 | vangaindrano
Processing Record 4 of Set 5 | new norfolk
Processing Record 5 of Set 5 | ajaccio
Processing Record 6 of Set 5 | upernavik
Processing Record 7 of Set 5 | bagotville
Processing Record 8 of Set 5 | mananara
Processing Record

Processing Record 28 of Set 8 | baglan
Processing Record 29 of Set 8 | sumenep
Processing Record 30 of Set 8 | vestmannaeyjar
Processing Record 31 of Set 8 | victoria point
Processing Record 32 of Set 8 | mnogovershinnyy
Processing Record 33 of Set 8 | moose factory
Processing Record 34 of Set 8 | brae
Processing Record 35 of Set 8 | samusu
City not found. Skipping...
Processing Record 36 of Set 8 | balkanabat
Processing Record 37 of Set 8 | poum
Processing Record 38 of Set 8 | det udom
Processing Record 39 of Set 8 | pijijiapan
Processing Record 40 of Set 8 | leningradskiy
Processing Record 41 of Set 8 | oskarshamn
Processing Record 42 of Set 8 | lillooet
Processing Record 43 of Set 8 | san cristobal
Processing Record 44 of Set 8 | sur
Processing Record 45 of Set 8 | mwingi
Processing Record 46 of Set 8 | noumea
Processing Record 47 of Set 8 | moberly
Processing Record 48 of Set 8 | evans
Processing Record 49 of Set 8 | pismo beach
Processing Record 50 of Set 8 | sao bento
Processing 

Processing Record 19 of Set 12 | aksarka
Processing Record 20 of Set 12 | korhogo
Processing Record 21 of Set 12 | oshnaviyeh
Processing Record 22 of Set 12 | rudbar
Processing Record 23 of Set 12 | garner
Processing Record 24 of Set 12 | colac
Processing Record 25 of Set 12 | bolungarvik
City not found. Skipping...
Processing Record 26 of Set 12 | avera
Processing Record 27 of Set 12 | hami
Processing Record 28 of Set 12 | rudnogorsk
Processing Record 29 of Set 12 | plettenberg bay
Processing Record 30 of Set 12 | mildura
Processing Record 31 of Set 12 | cairns
Processing Record 32 of Set 12 | quatre cocos
Processing Record 33 of Set 12 | bayshore gardens
Processing Record 34 of Set 12 | junin
Processing Record 35 of Set 12 | bolshoye sorokino
City not found. Skipping...
Processing Record 36 of Set 12 | rivadavia
Processing Record 37 of Set 12 | laguna
Processing Record 38 of Set 12 | jati
Processing Record 39 of Set 12 | grindavik
Processing Record 40 of Set 12 | helmsdale
Processing

In [19]:
len(city_data)

592

In [20]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Mongui,5.7215,-72.8491,41.40,93,100,1.83,CO,2022-08-23 07:07:15
1,Vardo,70.3705,31.1107,57.18,67,0,24.16,NO,2022-08-23 07:03:10
2,Mar Del Plata,-38.0023,-57.5575,47.08,96,100,6.91,AR,2022-08-23 07:07:16
3,Talara,-4.5772,-81.2719,64.51,80,17,13.13,PE,2022-08-23 07:07:17
4,Taunggyi,20.7833,97.0333,75.56,75,64,6.44,MM,2022-08-23 07:07:18
5,Cape Town,-33.9258,18.4232,67.12,68,0,4.61,ZA,2022-08-23 07:07:18
6,Buzmeyin,38.0507,58.2056,82.83,10,0,4.94,TM,2022-08-23 07:07:19
7,Oussouye,12.4850,-16.5469,77.43,86,83,4.50,SN,2022-08-23 07:07:19
8,Albany,42.6001,-73.9662,69.26,89,100,2.10,US,2022-08-23 07:02:37
9,Castro,-24.7911,-50.0119,45.97,98,25,3.40,BR,2022-08-23 07:07:20


In [23]:
new_column_order = ['City', 'Country', 'Date', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness', 'Wind Speed']
city_data_df = city_data_df[new_column_order]
city_data_df.head(10)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Mongui,CO,2022-08-23 07:07:15,5.7215,-72.8491,41.40,93,100,1.83
1,Vardo,NO,2022-08-23 07:03:10,70.3705,31.1107,57.18,67,0,24.16
2,Mar Del Plata,AR,2022-08-23 07:07:16,-38.0023,-57.5575,47.08,96,100,6.91
3,Talara,PE,2022-08-23 07:07:17,-4.5772,-81.2719,64.51,80,17,13.13
4,Taunggyi,MM,2022-08-23 07:07:18,20.7833,97.0333,75.56,75,64,6.44
5,Cape Town,ZA,2022-08-23 07:07:18,-33.9258,18.4232,67.12,68,0,4.61
6,Buzmeyin,TM,2022-08-23 07:07:19,38.0507,58.2056,82.83,10,0,4.94
7,Oussouye,SN,2022-08-23 07:07:19,12.4850,-16.5469,77.43,86,83,4.50
8,Albany,US,2022-08-23 07:02:37,42.6001,-73.9662,69.26,89,100,2.10
9,Castro,BR,2022-08-23 07:07:20,-24.7911,-50.0119,45.97,98,25,3.40


In [24]:
# Create the output file (CSV).
output_data_file = "weather_data/cities.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")